![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/36.Social_Determinant_of_Health_Models.ipynb)

# **Social Determinant of Health Models**

This notebook includes details about different kinds of pretrained models to extracts terminology related to Social Determinants of Health from various kinds of biomedical documents, together with examples of each type of model.

## Setup

In [ ]:
import json
import os

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

locals().update(license_keys)

os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [ ]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp_jsl.pretrained import InternalResourceDownloader

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

spark

Spark NLP Version : 5.0.0
Spark NLP_JSL Version : 5.0.0


## **List of Pretrained Models**

In [ ]:
df = pd.DataFrame()
for model_type in ['MedicalNerModel', 'GenericClassifierModel']:
    model_list = sorted(list(set([model[0] for model in InternalResourceDownloader.returnPrivateModels(model_type) if 'sdoh' in model[0]])))
    if len(model_list) > 0:
      if model_type == "MedicalNerModel":
        model_list = list(filter(lambda x: "wip" not in x, model_list))
      df = pd.concat([df, pd.DataFrame(model_list, columns = [model_type])], axis = 1)

df.fillna('')

MedicalNerModel  \
0                              ner_sdoh   
1         ner_sdoh_access_to_healthcare   
2          ner_sdoh_community_condition   
3                 ner_sdoh_demographics   
4   ner_sdoh_health_behaviours_problems   
5         ner_sdoh_income_social_status   
6                     ner_sdoh_mentions   
7                ner_sdoh_mentions_test   
8           ner_sdoh_social_environment   
9              ner_sdoh_substance_usage   
10                                        

                                              GenericClassifierModel  
0     genericclassifier_sdoh_alcohol_usage_binary_sbiobert_cased_mli  
1            genericclassifier_sdoh_alcohol_usage_sbiobert_cased_mli  
2         genericclassifier_sdoh_economics_binary_sbiobert_cased_mli  
3       genericclassifier_sdoh_housing_insecurity_sbiobert_cased_mli  
4       genericclassifier_sdoh_insurance_coverage_sbiobert_cased_mli  
5         genericclassifier_sdoh_insurance_status_sbiobert_cased_mli  
6           genericclassifier_sdoh_insurance_type_sbiobert_cased_mli  
7                      genericclassifier_sdoh_mental_health_clinical  
8   genericclassifier_sdoh_substance_usage_binary_sbiobert_cased_mli  
9            genericclassifier_sdoh_tobacco_usage_sbiobert_cased_mli  
10         genericclassifier_sdoh_under_treatment_sbiobert_cased_mli

## NER Models

The NER models from the list include different entity groups and levels of granularity.

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_detector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")\
    .setSplitChars(["-", "\/"])

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
    .setInputCols(["sentence","token"])\
    .setOutputCol("embeddings")

## ner_sdoh
ner_sdoh = MedicalNerModel.pretrained("ner_sdoh", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_sdoh")

ner_sdoh_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner_sdoh"]) \
    .setOutputCol("ner_chunk_sdoh") \

## ner_social_environment
ner_social_environment = MedicalNerModel.pretrained("ner_sdoh_social_environment_wip", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_social_environment")

ner_social_environment_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner_social_environment"]) \
    .setOutputCol("ner_chunk_social_environment") \

## ner_sdoh_mentions
ner_sdoh_mentions = MedicalNerModel.pretrained("ner_sdoh_mentions", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_sdoh_mentions")

ner_sdoh_mentions_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner_sdoh_mentions"]) \
    .setOutputCol("ner_chunk_ner_sdoh_mentions") \

ner_stages = [document_assembler,
    sentence_detector,
    tokenizer,
    word_embeddings,
    ner_sdoh,
    ner_sdoh_converter,
    ner_social_environment,
    ner_social_environment_converter,
    ner_sdoh_mentions,
    ner_sdoh_mentions_converter]

ner_pipeline = Pipeline(stages=ner_stages)

empty_data = spark.createDataFrame([[""]]).toDF("text")

ner_model = ner_pipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_sdoh download started this may take some time.
[OK!]
ner_sdoh_social_environment_wip download started this may take some time.
[OK!]
ner_sdoh_mentions download started this may take some time.
[OK!]


In [ ]:
ner_sdoh_labels = sorted(list(set([label.split('-')[-1] for label in ner_sdoh.getClasses() if label != 'O'])))

len(ner_sdoh_labels)

46

In [ ]:
label_df = pd.DataFrame()
for column in range((len(ner_sdoh_labels)//10)+1):
  label_df = pd.concat([label_df, pd.DataFrame(ner_sdoh_labels, columns = [''])[column*10:(column+1)*10].reset_index(drop= True)], axis = 1)

label_df.fillna('')

\
0        Access_To_Care               Employment    Hyperlipidemia   
1                   Age  Environmental_Condition      Hypertension   
2               Alcohol                 Exercise            Income   
3        Chidhood_Event            Family_Member  Insurance_Status   
4  Communicable_Disease         Financial_Status          Language   
5      Community_Safety          Food_Insecurity      Legal_Issues   
6                  Diet                   Gender    Marital_Status   
7            Disability        Geographic_Entity     Mental_Health   
8       Eating_Disorder   Healthcare_Institution           Obesity   
9             Education                  Housing     Other_Disease   

                                             
0  Other_SDoH_Keywords   Substance_Duration  
1     Population_Group  Substance_Frequency  
2      Quality_Of_Life   Substance_Quantity  
3       Race_Ethnicity        Substance_Use  
4      Sexual_Activity       Transportation  
5   Sexual_Orientation    Violence_Or_Abuse  
6              Smoking                       
7     Social_Exclusion                       
8       Social_Support                       
9    Spiritual_Beliefs

In [ ]:
ner_social_environment_labels = sorted(list(set([label.split('-')[-1] for label in ner_social_environment.getClasses() if label != 'O'])))

print(ner_social_environment_labels)

['Chidhood_Event', 'Social_Exclusion', 'Social_Support', 'Violence_Abuse_Legal']


In [ ]:
ner_sdoh_mentions_labels = sorted(list(set([label.split('-')[-1] for label in ner_sdoh_mentions.getClasses() if label != 'O'])))

print(ner_sdoh_mentions_labels)

['behavior_alcohol', 'behavior_drug', 'behavior_tobacco', 'sdoh_community', 'sdoh_economics', 'sdoh_education', 'sdoh_environment']


In [ ]:
sample_text_1 = '''Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels. Smith works as a cleaning assistant and cannot access health insurance or paid sick leave. She has a son, a student at college. Pt with likely long-standing depression. She is aware she needs rehab. Pt reports having her catholic faith as a means of support as well.  She has a long history of etoh abuse, beginning in her teens. She reports she has been a daily drinker for 30 years, most recently drinking beer daily. She smokes a pack of cigarettes a day. She had DUI in April and was due to court this week.'''

sample_text_2 = '''Medical history: Jane was born in a low - income household and experienced significant trauma during her childhood, including physical and emotional abuse.'''

sample_text_3 = '''Mr. Known lastname 9880 is a pleasant, cooperative gentleman with a long standing history (20 years) diverticulitis. He is married and has 3 children. He works in a bank. He denies any alcohol or intravenous drug use. He has been smoking for many years.'''

In [ ]:
data = spark.createDataFrame(pd.DataFrame([sample_text_1, sample_text_2, sample_text_3], columns = ['text']))

In [ ]:
results = ner_model.transform(data).collect()

In [ ]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

In [ ]:
from google.colab import widgets

t = widgets.TabBar(["ner_sdoh_mentions", "ner_sdoh_social_environment_wip", "ner_sdoh"])

with t.output_to(0):
    visualiser.display(results[2], label_col='ner_chunk_ner_sdoh_mentions')

with t.output_to(1):
    visualiser.display(results[1], label_col='ner_chunk_social_environment')

with t.output_to(2):
    visualiser.display(results[0], label_col='ner_chunk_sdoh')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Classification Model

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_embeddings = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

generic_classifier = GenericClassifierModel.pretrained("genericclassifier_sdoh_economics_binary_sbiobert_cased_mli", 'en', 'clinical/models')\
    .setInputCols(["features"])\
    .setOutputCol("classes")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_embeddings,
    features_asm,
    generic_classifier
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = pipeline.fit(empty_data)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
genericclassifier_sdoh_economics_binary_sbiobert_cased_mli download started this may take some time.
[OK!]


In [ ]:
sample_text = '''Patient works as a building inspector and remodeler. Married with 2 children. He is a current smoker, 1PPD for 25years. He drinks to beers/night, but has not had any alcohol in past 4 days. No IVDU.'''

In [ ]:
classification_data = spark.createDataFrame(pd.DataFrame([sample_text], columns = ['text']))

In [ ]:
classification_results = model.transform(classification_data)

In [ ]:
classification_results.select("text", "classes.result").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|text                                                                                                                                                                                                  |result|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Patient works as a building inspector and remodeler. Married with 2 children. He is a current smoker, 1PPD for 25years. He drinks to beers/night, but has not had any alcohol in past 4 days. No IVDU.|[True]|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Pretrained NER Profiling Pipelines

We can use pretrained NER profiling pipelines for exploring all the available pretrained NER models at once.

- `ner_profiling_sdoh` : Returns results for sdoh NER models.

For more examples, please check [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.2.Pretrained_NER_Profiling_Pipelines.ipynb).





<center><b>NER Profiling SDOH Model List</b>

|| | | |
|--------------|-----------------|-----------------|-----------------|
| ner_sdoh | ner_sdoh_social_environment_wip | ner_sdoh_mentions |
| ner_sdoh_demographics_wip | ner_sdoh_community_condition_wip | ner_sdoh_substance_usage_wip | ner_sdoh_access_to_healthcare_wip |
| ner_sdoh_health_behaviours_problems_wip | ner_sdoh_income_social_status_wip |



</center>

In [ ]:
from sparknlp.pretrained import PretrainedPipeline

sdoh_profiling_pipeline = PretrainedPipeline("ner_profiling_sdoh", "en", "clinical/models")

ner_profiling_sdoh download started this may take some time.
Approx size to download 1.6 GB
[OK!]


In [ ]:
text = """Smith is 55 years old, living in New York, a divorced Mexcian American woman with financial problems. She speaks Spanish and Portuguese. She lives in an apartment. She has been struggling with diabetes for the past 10 years and has recently been experiencing frequent hospitalizations due to uncontrolled blood sugar levels. Smith works as a cleaning assistant and cannot access health insurance or paid sick leave. She has a son, a student at college. Pt with likely long-standing depression. She is aware she needs rehab. Pt reports having her catholic faith as a means of support as well.  She has a long history of etoh abuse, beginning in her teens. She reports she has been a daily drinker for 30 years, most recently drinking beer daily. She smokes a pack of cigarettes a day. She had DUI in April and was due to court this week."""

In [ ]:
sdoh_result = sdoh_profiling_pipeline.fullAnnotate(text)[0]
sdoh_result.keys()

dict_keys(['ner_chunk_ner_sdoh_mentions', 'ner_chunk_sdoh_health_behaviours_problems', 'document', 'ner_chunk_social_environment', 'ner_chunk_sdoh_access_to_healthcare', 'ner_chunk_sdoh_substance_usage', 'ner_sdoh_mentions', 'ner_sdoh_community_condition', 'ner_chunk_ner_demographics', 'ner_sdoh_demographics', 'ner_chunk_sdoh_community_condition', 'ner_sdoh', 'ner_chunk_sdoh_income_social_status', 'ner_social_environment', 'ner_sdoh_access_to_healthcare', 'ner_sdoh_health_behaviours_problems', 'token', 'ner_sdoh_substance_usage', 'embeddings', 'ner_sdoh_income_social_status', 'ner_chunk_sdoh', 'sentence'])

In [ ]:
def get_token_results(light_result):

    tokens = [j.result for j in light_result["token"]]
    sentences = [j.metadata["sentence"] for j in light_result["token"]]
    begins = [j.begin for j in light_result["token"]]
    ends = [j.end for j in light_result["token"]]
    model_list = [ a for a in light_result.keys() if (a not in ["sentence", "token"] and "_chunks" not in a)]

    df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'token':tokens})

    for model_name in model_list:

        temp_df = pd.DataFrame(light_result[model_name])
        temp_df["jsl_label"] = temp_df.iloc[:,0].apply(lambda x : x.result)
        temp_df = temp_df[["jsl_label"]]

        # temp_df = get_ner_result(model_name)
        temp_df.columns = [model_name]
        df = pd.concat([df, temp_df], axis=1)

    # Filter columns to include only sentence, begin, end, token and all columns that start with 'ner_vop'
    filtered_df = df.loc[:, ['sentence', 'begin', 'end', 'token'] + [col for col in df.columns if col.startswith('ner_sdoh')]]

    return filtered_df

In [ ]:
get_token_results(sdoh_result)

sentence  begin  end  token ner_sdoh_mentions  \
0          0      0    4  Smith  B-sdoh_community   
1          0      6    7     is                 O   
2          0      9   10     55                 O   
3          0     12   16  years                 O   
4          0     18   20    old                 O   
..       ...    ...  ...    ...               ...   
159       12    817  818     to                 O   
160       12    820  824  court                 O   
161       12    826  829   this                 O   
162       12    831  834   week                 O   
163       12    835  835      .                 O   

    ner_sdoh_community_condition ner_sdoh_demographics ner_sdoh  \
0                              O                     O        O   
1                              O                     O        O   
2                              O                 B-Age    B-Age   
3                              O                 I-Age    I-Age   
4                              O                 I-Age    I-Age   
..                           ...                   ...      ...   
159                            O                     O        O   
160                            O                     O        O   
161                            O                     O        O   
162                            O                     O        O   
163                            O                     O        O   

    ner_sdoh_access_to_healthcare ner_sdoh_health_behaviours_problems  \
0                               O                                   O   
1                               O                                   O   
2                               O                                   O   
3                               O                                   O   
4                               O                                   O   
..                            ...                                 ...   
159                             O                                   O   
160                             O                                   O   
161                             O                                   O   
162                             O                                   O   
163                             O                                   O   

    ner_sdoh_substance_usage ner_sdoh_income_social_status  
0                          O                             O  
1                          O                             O  
2                          O                             O  
3                          O                             O  
4                          O                             O  
..                       ...                           ...  
159                        O                             O  
160                        O                             O  
161                        O                             O  
162                        O                             O  
163                        O                             O  

[164 rows x 12 columns]